<a href="https://colab.research.google.com/github/MicroprocessorX069/Stroke-Prediction-/blob/master/StrokePredictionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import drive
# drive.mount('/content/drive/')
 

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from classes5 import TabularDataset, FeedForwardNN
import torch.nn as nn
import torch.nn.functional as F
import os

In [76]:
path="/content/drive/My Drive/Projects/StrokePrediction/data/healthcare-dataset-stroke-data"
data= pd.read_csv(os.path.join(path,"train_2v.csv")).dropna()
data.shape

(29072, 12)

##Features segregation

In [0]:
categorical_features=["gender","ever_married","work_type","Residence_type",
                     "bmi","smoking_status"]
output_feature="stroke"

from sklearn.preprocessing import LabelEncoder
label_encoders={}
for cat_col in categorical_features:
  label_encoders[cat_col]=LabelEncoder()
  data[cat_col]=label_encoders[cat_col].fit_transform(data[cat_col])


##Creating object of class 'Tabular Dataset'

In [78]:
dataset=TabularDataset(data=data,cat_cols=categorical_features,
                      output_col=output_feature)
dataset[0]

[array([0]),
 array([3.0468e+04, 5.8000e+01, 1.0000e+00, 0.0000e+00, 8.7960e+01],
       dtype=float32),
 array([  1,   1,   2,   1, 261,   1])]

##Parameters

In [0]:
batchsize=200
dataloader=DataLoader(dataset,batchsize,shuffle=True,num_workers=1)

##Getting the dimensions for embeddings

In [80]:
cat_dims=[int(data[col].nunique()) for col in categorical_features]
cat_dims
emb_dims=[(x,min(128,(x+1)//2)) for x in cat_dims]
emb_dims

[(3, 2), (2, 1), (5, 3), (2, 1), (516, 128), (3, 2)]

##Creating an instance of model

In [0]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper parameters
no_of_cont=5
lin_layer_sizes=[128,64]
output_size=1
emb_dropout=0.04
lin_layer_dropouts=[0.001,0.01]
model= FeedForwardNN(emb_dims, no_of_cont, lin_layer_sizes,
                    output_size, emb_dropout,
                    lin_layer_dropouts).to(device)

##Training

In [82]:
no_of_epochs = 5
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
for epoch in range(no_of_epochs):
      for y, cont_x, cat_x in dataloader:
          
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
        y  = y.to(device)

        # Forward Pass
        preds = model(cont_x, cat_x)
        loss = criterion(preds, y)

        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

RuntimeError: ignored